In [0]:
df = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .option("multiLine", "true") \
    .option("quote", '"') \
    .option("escape", '"') \
    .csv("gs://bucket-autoai/complaints.csv")

In [0]:
pdf = df.toPandas()

In [0]:
pdf.info()

In [0]:
pdf.isnull().sum()

In [0]:
pdf.dropna(subset=['vin'], inplace=True)
pdf.dropna(subset=['manufacturer'], inplace=True)
pdf.dropna(subset=['components'], inplace=True)
pdf.dropna(subset=['model'], inplace=True)
pdf.dropna(subset=['products'], inplace=True)
pdf.dropna(subset=['modelYear'], inplace=True)
pdf.dropna(subset=['make'], inplace=True)


In [0]:
pdf.info()

In [0]:
pdf.isnull().sum()

In [0]:
pdf.duplicated().sum()

In [0]:
import pandas as pd
pdf['dateComplaintFiled'] = pd.to_datetime(pdf['dateComplaintFiled'])
pdf['dateOfIncident'] = pd.to_datetime(pdf['dateOfIncident'])



In [0]:
obj_cols = pdf.select_dtypes(include=["object"]).columns
pdf[obj_cols] = pdf[obj_cols].astype("string")


In [0]:
pdf.info()

In [0]:
pdf.head()

In [0]:
cols = ["products", "make", "model", "modelYear"]  

for c in cols:
    pdf[c] = (
        pdf[c].astype("string")
        .str.replace('""', '', regex=False)          
        .str.replace('"', '', regex=False)           
        .str.replace(r'[\[\]\{\}]', '', regex=True)
        .str.strip()
    )


In [0]:
pdf.head()

In [0]:
string_cols = pdf.select_dtypes(include=["string"]).columns

for c in string_cols:
    pdf[c] = (
        pdf[c]
        .fillna("")
        .str.lower()
        .str.strip()
        .str.replace(r"\s+", " ", regex=True)
        .str.replace(
            r"[^a-z0-9àâäçéèêëîïôöùûüÿñæœ\s\.\,\;\:\?\!\'\-\_/]",
            " ",
            regex=True,
        )
        .str.replace(r"\s+", " ", regex=True)
        .str.strip()
    )


In [0]:
pdf.head()

In [0]:
pdf["nb_characters"] = (
    pdf["summary"]
    .fillna("")
    .astype(str)
    .str.len()
)


In [0]:
pdf["nb_words"] = (
    pdf["summary"]
    .fillna("")
    .astype(str)
    .str.split()
    .str.len()
)


In [0]:
pdf["event_date"] = pdf["dateComplaintFiled"]


In [0]:
pdf["year"] = pdf["event_date"].dt.year
pdf["year_month"] = pdf["event_date"].dt.to_period("M").astype(str)
pdf["week"] = pdf["event_date"].dt.isocalendar().week.astype(int)
pdf["year_week"] = (
    pdf["event_date"].dt.isocalendar().year.astype(str)
    + "-W" +
    pdf["week"].astype(str).str.zfill(2)
)
pdf["quarter"] = pdf["event_date"].dt.to_period("Q").astype(str)


In [0]:
pdf.head()